In [1]:
# Install required libraries
import os

try:
    import selenium
except ImportError:
    os.system("pip install selenium")

try:
    import webdriver_manager
except ImportError:
    os.system("pip install webdriver-manager")

try:
    import pandas as pd
except ImportError:
    os.system("pip install pandas")

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [2]:
# Initialize Chrome with options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # Start maximized for better visibility
chrome_options.add_argument("--disable-infobars")  # Disable infobars
chrome_options.add_argument("--disable-extensions")  # Disable extensions for simplicity
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

def scroll_to_bottom(driver, pause_time=2):
    """
    Scroll to the bottom of the page to load dynamic content.
    """
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

driver.get("https://www.expedia.com/Hotel-Search?adults=2&children=&d1=2024-12-18&d2=2024-12-20&destination=Milan%20%28and%20vicinity%29%2C%20Lombardy%2C%20Italy&endDate=2024-12-20&latLong=45.47179%2C9.18617&mapBounds=&regionId=180012&rooms=1&semdtl=&sort=RECOMMENDED&startDate=2024-12-18&theme=&useRewards=false&userIntent=&pwaDialog=")
time.sleep(6)  # Wait for the page to load completely

# Extract hotel data
listing_container = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-stid="property-listing-results"]'))
)
hotel_links = listing_container.find_elements(By.CSS_SELECTOR, 'a[data-stid="open-hotel-information"]')
print(f"Found {len(hotel_links)} hotel links.")


Found 250 hotel links.


In [3]:
import pandas as pd

# Initialize a list to store all hotel data
hotel_data = []
location = "Milan, Italy"

for index, link in enumerate(hotel_links):
    try:
        href = link.get_attribute('href')
        print(f"\nProcessing link {index + 1}")

        # Open the link in a new tab using JavaScript
        driver.execute_script("window.open(arguments[0]);", href)
        time.sleep(3)  # Wait for the new tab to open

        # Switch to the new tab
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(15)  # Wait for the new page to load

        hotel_details = {"Location": location}

        # Extract Hotel Name
        try:
            hotel_name_element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "h1.uitk-heading.uitk-heading-3"))
            )
            hotel_details['Hotel Name'] = hotel_name_element.text.strip()
        except Exception as e:
            print(f"Error extracting hotel name: {e}")
            hotel_details['Hotel Name'] = None

        # Extract Stars
        try:
            stars_container = driver.find_element(By.CSS_SELECTOR, "div.uitk-rating.uitk-spacing.uitk-spacing-margin-blockstart-one")
            stars_count = len(stars_container.find_elements(By.TAG_NAME, "svg"))
            hotel_details['Stars'] = stars_count
        except Exception as e:
            print(f"Error extracting stars for hotel {index + 1}: {e}")
            hotel_details['Stars'] = None

        # Extract Rating
        try:
            rating_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "span.uitk-badge-base-large.uitk-badge-positive > span.uitk-badge-base-text")
                )
            )
            hotel_details['Rating'] = rating_element.text.strip()
        except Exception as e:
            print(f"Error extracting rating: {e}")
            hotel_details['Rating'] = None

        # Extract Review Keyword
        try:
            review_summary_container = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-stid='content-hotel-reviewsummary']"))
            )
            review_keyword_element = review_summary_container.find_element(
                By.CSS_SELECTOR, "div.uitk-text.uitk-type-500.uitk-type-medium.uitk-text-default-theme"
            )
            review_keyword = review_keyword_element.text.strip()
            hotel_details['Review Keyword'] = review_keyword
            print(f"Extracted Review Keyword: {review_keyword}")
        except Exception as e:
            print(f"Error extracting review keyword: {e}")
            hotel_details['Review Keyword'] = None

        # Extract Services
        try:
            services_list = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "ul.uitk-typelist[data-stid='sp-content-list']"))
            )

            services_lis = services_list.find_elements(By.CSS_SELECTOR, "li[data-stid^='sp-content-item-']")
            service_count = len(services_lis)
            print(f"Number of services: {service_count}")

            services = []
            for i in range(service_count):
                try:
                    service_span = driver.find_element(
                        By.CSS_SELECTOR,
                        f"ul.uitk-typelist[data-stid='sp-content-list'] li[data-stid='sp-content-item-{i}'] span.uitk-text.uitk-type-300.uitk-text-default-theme.uitk-layout-flex-item"
                    )
                    services.append(service_span.text.strip())
                except Exception as e:
                    print(f"Error extracting service {i}: {e}")
                    services.append(None)

            hotel_details['Services'] = services
        except Exception as e:
            print(f"Error extracting services: {e}")
            hotel_details['Services'] = []

        # Extract Property Offers
        try:
            property_offers_container = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-stid="section-room-list"]'))
            )

            property_offers = property_offers_container.find_elements(By.CSS_SELECTOR, 'div[data-stid^="property-offer-"]')

            offers = []
            for offer_index, offer in enumerate(property_offers, start=1):
                offer_details = {}

                try:
                    room_name_element = offer.find_element(By.CSS_SELECTOR, "h3.uitk-heading.uitk-heading-6")
                    offer_details['Room Name'] = room_name_element.text.strip()
                except Exception as e:
                    print(f"Error extracting room name for offer {offer_index}: {e}")
                    offer_details['Room Name'] = None

                try:
                    price_summary_div = offer.find_element(By.CSS_SELECTOR, 'div[data-stid="price-summary"]')
                    room_charges_element = price_summary_div.find_element(
                        By.CSS_SELECTOR,
                        "div.uitk-text.uitk-type-500.uitk-type-medium.uitk-text-emphasis-theme"
                    )
                    offer_details['Room Charges'] = room_charges_element.text.strip()
                except Exception as e:
                    print(f"Error extracting room charges for offer {offer_index}: {e}")
                    offer_details['Room Charges'] = None

                offers.append(offer_details)

            hotel_details['Property Offers'] = offers
        except Exception as e:
            print(f"Error extracting property offers: {e}")
            hotel_details['Property Offers'] = []

        # Add hotel details to list
        hotel_data.append(hotel_details)
        print(f"Extracted Hotel Data: {hotel_details}")

        # Close the new tab
        driver.close()

        # Switch back to the main tab
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(1)

    except Exception as e:
        print(f"Error processing link {index + 1}: {e}")
        continue

# Export hotel data to a CSV file
df = pd.DataFrame(hotel_data)

# Normalize property offers if necessary
if 'Property Offers' in df.columns:
    df['Property Offers'] = df['Property Offers'].apply(lambda x: ', '.join(str(offer) for offer in x) if isinstance(x, list) else x)

# Save to CSV
output_filename = f"hotel_data_{location.replace(',', '').replace(' ', '_')}.csv"
df.to_csv(output_filename, index=False)
print(f"Data exported to {output_filename}")


Processing link 1
Extracted Review Keyword: Wonderful
Number of services: 6
Extracted Hotel Data: {'Location': 'Milan, Italy', 'Hotel Name': 'Palazzo Cordusio, a Gran Melia Hotel', 'Stars': 5, 'Rating': '9.0', 'Review Keyword': 'Wonderful', 'Services': ['Spa', 'Bar', 'Parking available', 'Breakfast available', '24/7 front desk', 'Air conditioning'], 'Property Offers': [{'Room Name': 'Premier Room', 'Room Charges': '$817'}, {'Room Name': 'Premier Room, Balcony', 'Room Charges': '$841'}, {'Room Name': 'Room (Supreme)', 'Room Charges': '$922'}, {'Room Name': 'Junior Room (Suite)', 'Room Charges': '$1,141'}, {'Room Name': 'Junior Room (Suite 2+1)', 'Room Charges': '$1,162'}, {'Room Name': 'Premium Room, 2 Bedrooms (Connecting)', 'Room Charges': '$1,492'}, {'Room Name': 'Suite, 1 Bedroom (Master)', 'Room Charges': '$1,505'}, {'Room Name': 'Suite, 1 Bedroom (Master, 2+1)', 'Room Charges': '$1,526'}, {'Room Name': 'Suite, 1 Bedroom, Balcony (Master)', 'Room Charges': '$1,586'}, {'Room Name':


Processing link 5
Extracted Review Keyword: Excellent
Number of services: 6
Error extracting room charges for offer 8: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x005C34A3+25059]
	(No symbol) [0x0054CEA4]
	(No symbol) [0x0042BEC3]
	(No symbol) [0x0046FD86]
	(No symbol) [0x0046FFCB]
	(No symbol) [0x00465B01]
	(No symbol) [0x00491F44]
	(No symbol) [0x00465A24]
	(No symbol) [0x00492194]
	(No symbol) [0x004AB51E]
	(No symbol) [0x00491C96]
	(No symbol) [0x00463FAC]
	(No symbol) [0x00464F3D]
	GetHandleVerifier [0x008B5613+3113811]
	GetHandleVerifier [0x008CA2DA+3199002]
	GetHandleVerifier [0x008C2AB2+3168242]
	GetHandleVerifier [0x00663310+680016]
	(No symbol) [0x005557ED]
	(No symbol) [0x00552A98]


Processing link 7
Extracted Review Keyword: Excellent
Number of services: 6
Error extracting room charges for offer 7: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x005C34A3+25059]
	(No symbol) [0x0054CEA4]
	(No symbol) [0x0042BEC3]
	(No symbol) [0x0046FD86]
	(No symbol) [0x0046FFCB]
	(No symbol) [0x00465B01]
	(No symbol) [0x00491F44]
	(No symbol) [0x00465A24]
	(No symbol) [0x00492194]
	(No symbol) [0x004AB51E]
	(No symbol) [0x00491C96]
	(No symbol) [0x00463FAC]
	(No symbol) [0x00464F3D]
	GetHandleVerifier [0x008B5613+3113811]
	GetHandleVerifier [0x008CA2DA+3199002]
	GetHandleVerifier [0x008C2AB2+3168242]
	GetHandleVerifier [0x00663310+680016]
	(No symbol) [0x005557ED]
	(No symbol) [0x00552A98]


Processing link 9
Error extracting stars for hotel 9: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.uitk-rating.uitk-spacing.uitk-spacing-margin-blockstart-one"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x005C34A3+25059]
	(No symbol) [0x0054CEA4]
	(No symbol) [0x0042BEC3]
	(No symbol) [0x0046FD86]
	(No symbol) [0x0046FFCB]
	(No symbol) [0x004AD952]
	(No symbol) [0x00491F44]
	(No symbol) [0x004AB51E]
	(No symbol) [0x00491C96]
	(No symbol) [0x00463FAC]
	(No symbol) [0x00464F3D]
	GetHandleVerifier [0x008B5613+3113811]
	GetHandleVerifier [0x008CA2DA+3199002]
	GetHandleVerifier [0x008C2AB2+3168242]
	GetHandleVerifier [0x00663310+680016]
	(No symbol) [0x005557ED]
	(No symbol) [0x00552A98]
	(No symbol) [0x00552C35]
	(No symbol) [0x00545890]
	BaseThreadInitThunk [0x76F6FCC


Processing link 11
Extracted Review Keyword: Wonderful
Number of services: 6
Error extracting room charges for offer 4: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x005C34A3+25059]
	(No symbol) [0x0054CEA4]
	(No symbol) [0x0042BEC3]
	(No symbol) [0x0046FD86]
	(No symbol) [0x0046FFCB]
	(No symbol) [0x00465B01]
	(No symbol) [0x00491F44]
	(No symbol) [0x00465A24]
	(No symbol) [0x00492194]
	(No symbol) [0x004AB51E]
	(No symbol) [0x00491C96]
	(No symbol) [0x00463FAC]
	(No symbol) [0x00464F3D]
	GetHandleVerifier [0x008B5613+3113811]
	GetHandleVerifier [0x008CA2DA+3199002]
	GetHandleVerifier [0x008C2AB2+3168242]
	GetHandleVerifier [0x00663310+680016]
	(No symbol) [0x005557ED]
	(No symbol) [0x00552A98

...


Processing link 237
Extracted Review Keyword: Excellent
Number of services: 6
Extracted Hotel Data: {'Location': 'Milan, Italy', 'Hotel Name': 'WorldHotel Casati 18', 'Stars': 4, 'Rating': '8.6', 'Review Keyword': 'Excellent', 'Services': ['Gym', 'Bar', 'Breakfast available', 'Room service', '24/7 front desk', 'Housekeeping'], 'Property Offers': [{'Room Name': 'Deluxe Room, 2 Twin Beds, Non Smoking', 'Room Charges': '$189'}, {'Room Name': 'Executive Room, 1 King Bed, Non Smoking', 'Room Charges': '$213'}, {'Room Name': 'Deluxe Room, 1 King Bed, Non Smoking', 'Room Charges': '$205'}]}

Processing link 238
Error extracting stars for hotel 238: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.uitk-rating.uitk-spacing.uitk-spacing-margin-blockstart-one"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktra

Error extracting room charges for offer 3: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x005C34A3+25059]
	(No symbol) [0x0054CEA4]
	(No symbol) [0x0042BEC3]
	(No symbol) [0x0046FD86]
	(No symbol) [0x0046FFCB]
	(No symbol) [0x00465B01]
	(No symbol) [0x00491F44]
	(No symbol) [0x00465A24]
	(No symbol) [0x00492194]
	(No symbol) [0x004AB51E]
	(No symbol) [0x00491C96]
	(No symbol) [0x00463FAC]
	(No symbol) [0x00464F3D]
	GetHandleVerifier [0x008B5613+3113811]
	GetHandleVerifier [0x008CA2DA+3199002]
	GetHandleVerifier [0x008C2AB2+3168242]
	GetHandleVerifier [0x00663310+680016]
	(No symbol) [0x005557ED]
	(No symbol) [0x00552A98]
	(No symbol) [0x00552C35]
	(No symbol) [0x00545890]
	BaseThreadInitThunk [0x

Error extracting room charges for offer 16: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x005C34A3+25059]
	(No symbol) [0x0054CEA4]
	(No symbol) [0x0042BEC3]
	(No symbol) [0x0046FD86]
	(No symbol) [0x0046FFCB]
	(No symbol) [0x00465B01]
	(No symbol) [0x00491F44]
	(No symbol) [0x00465A24]
	(No symbol) [0x00492194]
	(No symbol) [0x004AB51E]
	(No symbol) [0x00491C96]
	(No symbol) [0x00463FAC]
	(No symbol) [0x00464F3D]
	GetHandleVerifier [0x008B5613+3113811]
	GetHandleVerifier [0x008CA2DA+3199002]
	GetHandleVerifier [0x008C2AB2+3168242]
	GetHandleVerifier [0x00663310+680016]
	(No symbol) [0x005557ED]
	(No symbol) [0x00552A98]
	(No symbol) [0x00552C35]
	(No symbol) [0x00545890]
	BaseThreadInitThunk [0


Processing link 244
Extracted Review Keyword: Very good
Number of services: 6
Extracted Hotel Data: {'Location': 'Milan, Italy', 'Hotel Name': 'Leonardo Hotel Milan City Center', 'Stars': 4, 'Rating': '8.4', 'Review Keyword': 'Very good', 'Services': ['Bar', 'Parking available', 'Gym', 'Breakfast available', 'Pet friendly', 'Room service'], 'Property Offers': [{'Room Name': 'Comfort Room', 'Room Charges': '$144'}, {'Room Name': 'Superior Room', 'Room Charges': '$147'}, {'Room Name': 'Double Comfort', 'Room Charges': '$149'}, {'Room Name': 'Double Or Twin Superior', 'Room Charges': '$152'}, {'Room Name': 'Triple Superior', 'Room Charges': '$174'}, {'Room Name': 'Suite, 1 King Bed', 'Room Charges': '$211'}]}

Processing link 245
Extracted Review Keyword: Exceptional
Number of services: 6
Error extracting room charges for offer 6: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: chrome=131.0.6778.14

Extracted Review Keyword: Very good
Number of services: 6
Error extracting room charges for offer 7: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x005C34A3+25059]
	(No symbol) [0x0054CEA4]
	(No symbol) [0x0042BEC3]
	(No symbol) [0x0046FD86]
	(No symbol) [0x0046FFCB]
	(No symbol) [0x00465B01]
	(No symbol) [0x00491F44]
	(No symbol) [0x00465A24]
	(No symbol) [0x00492194]
	(No symbol) [0x004AB51E]
	(No symbol) [0x00491C96]
	(No symbol) [0x00463FAC]
	(No symbol) [0x00464F3D]
	GetHandleVerifier [0x008B5613+3113811]
	GetHandleVerifier [0x008CA2DA+3199002]
	GetHandleVerifier [0x008C2AB2+3168242]
	GetHandleVerifier [0x00663310+680016]
	(No symbol) [0x005557ED]
	(No symbol) [0x00552A98]
	(No symbol) [0x00